## create DataFrame from ICE pdf settlement file
See https://www.theice.com/marketdata/reports/166

In [14]:
import pandas as pd
import numpy as np
import re
import pathlib
import datetime
import re


In [42]:
year = 2021
month = 11
day = 11
folder = f'{pathlib.Path.home()}/downloads'

In [28]:
! bash {pathlib.Path.home()}/pdfjar_extract_text.sh {folder} B_{year}_{month}_{day}

In [43]:
ice_brent_txt = open(f'{folder}/B_{year}_{month}_{day}.txt').readlines()
cols = [
    'commod','mmmyy','strike','pc','delta',
    'open','high','low','close','settle','price_change',
    'volume','oi','oi_change','exer','block_volume','eoo','spread_volume'
]
csv_lines = [
#     ','.join(goodline[0].split(' ')) for goodline in 
#     goodline[0].replace(' ',',').split(',') for goodline in 
    goodline[0].split(' ') for goodline in 
    [
        re.findall("B [A-Z][a-z]{2}[2-3][0-9] [1-9][0-9]{1,4}\.[0-9]{2,4} [PC] .+",line)
        for line in ice_brent_txt
    ]
    if len(goodline)>0
]

csv_lines_sizes = list(set([len(arr) for arr in csv_lines]))
min_line_size = min(csv_lines_sizes)
max_line_size = max(csv_lines_sizes)

if max_line_size == 18:
    csv_lines = [
        line if len(line)==max_line_size else line[0:5] + [line[5],line[5],line[5],line[5],line[5]] + line[6:]
        for line in csv_lines
    ]


df_brent_crude_ice = pd.DataFrame(
    csv_lines,
    columns=cols
)

df_brent_crude_ice = df_brent_crude_ice.fillna(0)
poss_float_cols = [
    'strike','delta',
    'open','high','low','close','settle','price_change',
    'volume','oi','oi_change','exer','block_volume','eoo','spread_volume'
]
float_cols = [c for c in poss_float_cols if c in df_brent_crude_ice.columns.values]
for col in float_cols:
    df_brent_crude_ice[col] = df_brent_crude_ice[col].str.replace(',','').astype(float) 
df_brent_crude_ice

,commod,mmmyy,strike,pc,delta,open,high,low,close,settle,price_change,volume,oi,oi_change,exer,block_volume,eoo,spread_volume
0,B,Jan22,34.00,C,0.9996,48.87,48.87,48.87,48.87,48.87,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,B,Jan22,39.00,C,0.9992,43.87,43.87,43.87,43.87,43.87,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,B,Jan22,47.00,C,0.9980,35.88,35.88,35.88,35.88,35.88,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,B,Jan22,49.00,C,0.9975,33.88,33.88,33.88,33.88,33.88,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,B,Jan22,49.75,C,0.9973,33.13,33.13,33.13,33.13,33.13,0.23,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,B,Dec24,45.00,P,-0.1590,4.74,4.74,4.74,4.74,4.74,-0.27,0.0,750.0,0.0,0.0,0.0,0.0,0.0
1796,B,Dec24,60.00,P,-0.3018,9.79,9.79,9.79,9.79,9.79,-0.32,0.0,1680.0,0.0,0.0,0.0,0.0,0.0
1797,B,Dec25,20.00,P,-0.0299,0.88,0.88,0.88,0.88,0.88,-0.13,0.0,390.0,0.0,0.0,0.0,0.0,0.0
1798,B,Dec25,40.00,P,-0.1377,4.50,4.50,4.50,4.50,4.50,-0.26,0.0,1170.0,0.0,0.0,0.0,0.0,0.0


In [25]:
c_1 = df_brent_crude_ice.mmmyy=='Dec22'
c_2 = df_brent_crude_ice.pc == 'C'
c_3 = df_brent_crude_ice['strike'] >= 90
c_4 = df_brent_crude_ice.volume>=500

# c_all = c_1 & c_2 & c_3 & c_4
c_all =  c_4
df_brent_crude_ice[c_all].sort_values('volume',ascending=False)

,commod,mmmyy,strike,pc,delta,open,high,low,close,settle,price_change,volume,oi,oi_change,exer,block_volume,eoo,spread_volume
394,B,Mar22,95.0,C,0.1692,1.04,1.15,1.00,1.02,1.06,-0.04,15133.0,12126.0,-4713.0,0.0,5100.0,0.0,14646.0
385,B,Mar22,90.0,C,0.2704,1.84,1.84,1.79,1.79,1.92,-0.03,10999.0,21157.0,4766.0,0.0,2500.0,0.0,10705.0
1585,B,Jun22,60.0,P,-0.1479,2.20,2.28,2.10,2.10,2.08,-0.17,6285.0,29258.0,4792.0,0.0,4850.0,0.0,5435.0
671,B,Jun22,93.0,C,0.2687,2.66,2.66,2.66,2.66,2.66,0.04,6000.0,6072.0,6000.0,0.0,6000.0,0.0,6000.0
842,B,Dec22,250.0,C,0.0060,0.06,0.06,0.06,0.06,0.06,0.00,5000.0,5000.0,5000.0,0.0,5000.0,0.0,5000.0
277,B,Feb22,100.0,C,0.0523,0.22,0.25,0.20,0.20,0.21,-0.01,3517.0,13610.0,-488.0,0.0,0.0,0.0,2409.0
132,B,Jan22,90.0,C,0.1063,0.28,0.32,0.28,0.30,0.27,-0.04,3376.0,24173.0,452.0,0.0,0.0,0.0,2640.0
666,B,Jun22,90.0,C,0.3187,3.21,3.21,3.21,3.21,3.35,0.05,3044.0,17878.0,1236.0,0.0,2050.0,0.0,2994.0
260,B,Feb22,90.0,C,0.2131,1.01,1.02,0.97,0.99,1.05,-0.04,2999.0,19066.0,268.0,0.0,500.0,0.0,2760.0
1290,B,Mar22,65.0,P,-0.1146,1.10,1.10,1.02,1.02,0.98,-0.12,2980.0,15355.0,2750.0,0.0,2750.0,0.0,2869.0


In [6]:
df_brent_crude_ice.to_csv(f'{folder}/B_{year}_{month}_{day}.csv',index=False)